# Exploratory Analysis and cleaning
This notebook will serve as an interation for cleaning and verifying fixes in the OSM data. The MongoDB test database will be used as it's much more performant than iterating over the JSON file

In [3]:
# Import Mongodb Driver
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')
db = client.test

## Inspecting some nodes in greater detail
Random documents will be selected to for manual inspection to understand the OSM structure in practice.

In [18]:
#Get Number of document
num_documents = db.la_map.find().count()

#Get random list
import random
random.seed(2)

random_list = [random.randrange(0,num_documents) for num in range(10)]
random_list

[474387,
 768268,
 711957,
 3028769,
 1418376,
 5618105,
 2584835,
 2110433,
 5083028,
 1780220]

In [19]:
import pprint
c = db.la_map.find()
for document_num in random_list:
    pprint.pprint(c[document_num])
    print("")

{'_id': ObjectId('5666f24db854b14d56c2a8bb'),
 'address': {},
 'created': {'changeset': '3484407',
             'timestamp': '2009-12-29T20:07:33Z',
             'uid': '147510',
             'user': 'woodpeck_fixbot',
             'version': '2'},
 'id': '95445789',
 'node_refs': [],
 'pos': [34.312752, -118.852562],
 'type': 'node'}

{'_id': ObjectId('5666f253b854b14d56c724b4'),
 'address': {},
 'created': {'changeset': '12753187',
             'timestamp': '2012-08-16T17:16:32Z',
             'uid': '53073',
             'user': 'Aaron Lidman',
             'version': '5'},
 'id': '122759371',
 'node_refs': [],
 'pos': [33.8271577, -117.7824221],
 'type': 'node'}

{'_id': ObjectId('5666f252b854b14d56c648bd'),
 'address': {},
 'created': {'changeset': '15790934',
             'timestamp': '2013-04-19T23:17:54Z',
             'uid': '53073',
             'user': 'Aaron Lidman',
             'version': '4'},
 'id': '122627522',
 'node_refs': [],
 'pos': [33.8392396, -117.7650139],
 'ty

Most of these results are fairly boring. The node just tells us is position and when it was created. As is such I'd like to take a look at distribution for the length of records from the LA OSM file.

### Pymongo Limitation
This actually would have been much easier if I was using the MongoDB terminal client. Pymongo is notably missing the forEach method which allows for a javascript style query which has great flexbility when working with the objects. However it eventually works out as we will be using the robust methods in Pandas to provide a deeper analysis of distribution of document lengths

In [25]:
document_lengths = []
for document in c:
    document_lengths.append([document["id"], len(document)])
    
document_lengths[:5]

[['653661', 7], ['653678', 7], ['653688', 7], ['653689', 7], ['653690', 7]]

In [27]:
import pandas as pd
df = pd.DataFrame(document_lengths, columns = ["id", "Length"])
df.shape

(5953757, 2)

In [34]:
df.head()

,id,Length
0,653661,7
1,653678,7
2,653688,7
3,653689,7
4,653690,7


In [32]:
#del document_lengths
df["Length"].describe()

count    5953757.000000
mean           7.625952
std            2.237075
min            7.000000
25%            7.000000
50%            7.000000
75%            7.000000
max           46.000000
Name: Length, dtype: float64

The majority of nodes have 7 values, at least over 75%. This is consistenet with what we saw above. Checking to see how many have above seven values.

In [33]:
df.ix[df["Length"] > 7 ,"Length"].describe()

count    839765.000000
mean         11.437871
std           4.308606
min           8.000000
25%           8.000000
50%           9.000000
75%          15.000000
max          46.000000
Name: Length, dtype: float64

Let's get random items from the MongoDB database that have items over 7. Since I iterated over the entire MongoDB file each dataframe row should correspond to it's matching instance in MongoDB.

In [56]:
c = db.la_map.find()

for document_num in df.ix[df["Length"] > 9 ,"Length"].sample(10).index:
    pprint.pprint(c[int(document_num)])
    print("")

{'_id': ObjectId('5666f2d3b854b14d56108edd'),
 'access': 'private',
 'address': {},
 'created': {'changeset': '3286993',
             'timestamp': '2009-12-04T09:47:44Z',
             'uid': '82317',
             'user': 'AM909',
             'version': '1'},
 'highway': 'service',
 'id': '45545897',
 'node_refs': ['580751571', '580751626'],
 'oneway': 'yes',
 'pos': ['Lat', 'Lon'],
 'source': 'survey;image;usgs_imagery',
 'source_ref': 'AM909_DSCQ1192',
 'type': 'way'}

{'_id': ObjectId('5666f2d8b854b14d56126200'),
 'address': {},
 'building': 'yes',
 'created': {'changeset': '11328457',
             'timestamp': '2012-04-17T01:25:42Z',
             'uid': '313196',
             'user': 'The Temecula Mapper',
             'version': '1'},
 'id': '159934214',
 'node_refs': ['1719987667',
               '1719979329',
               '1719979328',
               '1719987666',
               '1719987667'],
 'pos': ['Lat', 'Lon'],
 'source': 'bing_imagery',
 'type': 'way'}

{'_id': ObjectId

Again nothing stands out here. Let's take a look at the keys themselves

In [57]:
c[0].keys()

dict_keys(['type', 'address', 'node_refs', 'pos', '_id', 'created', 'id'])

In [80]:
c = db.la_map.find()
tag_count = {}
for document in c:
    for key in document.keys():
        if tag_count.get(key):
            tag_count[key] +=1
        else:
            tag_count[key] = 1   

df_keycount = pd.Series(tag_count)

In [81]:
df_keycount.head().sort_index()

3dr:type       1
4wd_only       3
AIN         9713
ALAND        332
APN           49
dtype: int64

In [82]:
df_keycount.to_csv("Attributes.csv")

In [84]:
df_keycount.describe()

count       1707.000000
mean       26598.169303
std       380915.788686
min            1.000000
25%            1.000000
50%            5.000000
75%           69.000000
max      5953758.000000
dtype: float64

The majority of tags are used less than 5 times. When looking through the tag data manually it is notable that some tags are essentially the same thing. For instance **Wifi_Access** and **Internet** indicate whether internet access is available but with two tags it becomes more difficult to find all places that have internet access